To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### CONFIGURATION

In [1]:
# if you run this notebook on vs code for windows 11 follow the documentation, do not install unsloth directly cause it automatically install torch only for cpu, when you change it and you start messing up everything

# follow this step:

# 1) install the right version of pytorch for your cuda ( in my case is 118+cuda )
# 2) open you command line (powershell doesnt work with this) and run:
# pip install "unsloth[windows] @ git+https://github.com/unslothai/unsloth.git"
# plus you will need some packages like ipywidgets but should be not a problem

In [ ]:
import pandas as pd
import json
import ast
import sys
import torch
import os
import pandas as pd
from datasets import Dataset
import unsloth
from unsloth import FastLanguageModel
import numpy as np

# you have to install ipywidgets e python-dotenv
from huggingface_hub import login
import os
login(token = os.getenv("HF_TOKEN"))


In [ ]:
# Imposta l'encoding UTF-8
os.environ["PYTHONIOENCODING"] = "utf-8"
os.environ['PYTHONUTF8'] = '1'

In [ ]:
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA disponibile: {torch.cuda.is_available()}')
print(f'Numero GPU: {torch.cuda.device_count()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'CUDA version: {torch.version.cuda}')

In [ ]:
df = pd.read_csv('DatasetConReviewsFinale.csv', encoding='utf-8')

### Unsloth

In [7]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = os.getenv("HF_TOKEN") # use one if using gated models like meta-llama/Llama-2-7b-hf

)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    NVIDIA GeForce RTX 4070. Num GPUs = 1. Max memory: 11.994 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.1+cu118. CUDA: 8.9. CUDA Toolkit: 11.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.7.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [ ]:
# functions to clean the dataset

def is_valid_structure(review_str):
    """Check if review has valid structure with required scores"""
    try:
        data_dict = ast.literal_eval(review_str)
        if not isinstance(data_dict, dict):
            return False
        
        required_keys = {'score_correttezza_grammaticale', 'score_hallucination', 
                        'score_toxicity', 'score_engagement', 'description'}
        if not required_keys.issubset(data_dict.keys()):
            return False
        
        # Check if all scores are in range 1-5
        score_keys = ['score_correttezza_grammaticale', 'score_hallucination', 
                     'score_toxicity', 'score_engagement']
        for key in score_keys:
            score = data_dict[key]
            if not isinstance(score, int) or not (1 <= score <= 5):
                return False
        
        return True
    
    except (ValueError, SyntaxError, TypeError):
        return False

def clean_dataset(df, column_name='Reviews'):
    """Clean dataset keeping only valid reviews"""
    valid_mask = df[column_name].apply(is_valid_structure)
    cleaned_df = df[valid_mask].copy()
    removed_df = df[~valid_mask].copy()
    
    print(f"Original: {len(df)}, Cleaned: {len(cleaned_df)}, Removed: {len(removed_df)}")
    return cleaned_df, removed_df

def calculate_average_scores(df, column_name='reviews'):
    """Calculate average scores from reviews"""
    scores = {'score_correttezza_grammaticale': [], 'score_hallucination': [], 
              'score_toxicity': [], 'score_engagement': []}
    
    for review_str in df[column_name]:
        review_dict = ast.literal_eval(review_str)
        for score_key in scores.keys():
            scores[score_key].append(review_dict[score_key])
    
    average_scores = {key: sum(values)/len(values) for key, values in scores.items()}
    return average_scores

def count_scores_distribution(df, column_name='reviews'):
    """Count distribution of scores (1-5) for each category"""
    from collections import defaultdict, Counter
    
    score_distributions = defaultdict(list)
    
    for review_str in df[column_name]:
        review_dict = ast.literal_eval(review_str)
        for score_key in ['score_correttezza_grammaticale', 'score_hallucination', 
                         'score_toxicity', 'score_engagement']:
            score_distributions[score_key].append(review_dict[score_key])
    
    # Count occurrences for each score category
    distributions = {}
    for category, scores in score_distributions.items():
        distributions[category] = dict(Counter(scores))
    
    return distributions

In [12]:
cleaned_df, rejected_df = clean_dataset(df)

Original: 1649, Cleaned: 1598, Removed: 51


In [13]:
test_set = pd.DataFrame(rejected_df['Text']).reset_index(drop=True)
train_set = cleaned_df.reset_index(drop=True)

In [ ]:
# introduci la colonna instruction che verrà utilizzata per il fine tuning
instruction = "Analyze the given text and provide an evaluation in the same format as the provided example."
instructions = [instruction] * train_set.shape[0]
train_set['Instruction'] = instructions

In [ ]:
# rinomino le features per adattarmi al formato alpaca che consiglia unsloth, quindi avrò una colonna instruction, una text e una output, risistemo anche gli indici che si erano sminchiati durante la pulizia
train_set = train_set[['Instruction', 'Text', 'Reviews']].rename(columns={
    'Instruction': 'instruction',
    'Text': 'input',
    'Reviews': 'output'
})

train_set['output'] = train_set['output'].astype(str)

# converto in un hugging face dataset come nell'esempio
final_df = Dataset.from_pandas(train_set)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 1598
})

In [ ]:
# cella presa dal codice, modifica il dataset e lo mette in formato alpaca

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
train_set_to_pass = final_df.map(formatting_prompts_func, batched = True,)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTConfig, SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_set_to_pass,              # Train split
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = True, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        dataset_num_proc=1,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        max_steps = -1,
        learning_rate = 1e-3,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "tensorboard", # Use this for WandB etc
    ),
)

In [24]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4070. Max memory = 11.994 GB.
5.75 GB of memory reserved.


In [27]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,598 | Num Epochs = 3 | Total steps = 600
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.606700
2,2.679300
3,2.650900
4,2.678800
5,2.397400
6,2.597400
7,2.381400
8,2.411400
9,2.545000
10,2.449200


In [28]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3685.567 seconds used for training.
61.43 minutes used for training.
Peak reserved memory = 14.855 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 123.854 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [33]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "is this sentence engaging?", # instruction
        "Hello, I'm here to build this new podcast, you can join me and help!", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, temperature = 0.01)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nis this sentence engaging?\n\n### Input:\nHello, I\'m here to build this new podcast, you can join me and help!\n\n### Response:\n{\'score\': 2, \'description\': "The transcript is a brief announcement with minimal context, lacking compelling storytelling or emotional depth. It attempts to create curiosity but does so poorly by using a generic hook (\'Hello\') and vague language (\'build this new podcast\'). The conversational dynamics are absent as it\'s a solo']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [78]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
    [
    alpaca_prompt.format(
        "is this sentence engaging?", # instruction
        "hello this is my new podcast", # input
        "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
answer = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, temperature = 0.01)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
is this sentence engaging?

### Input:
hello this is my new podcast

### Response:
{'score': 1, 'description': "The transcript lacks compelling storytelling, emotional depth, or interesting hooks to maintain listener interest. It's a simple statement without any narrative flow, emotional connection, or curiosity-inducing elements."}<|end_of_text|>


In [ ]:
# alpaca_prompt = Copied from above
answers = []
for text in test_set['Text']:
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            "is this sentence engaging?", # instruction
            test_set['Text'], # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    answer = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, temperature = 0.01)
    answers.append(answer)
    
answers


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
# model.save_pretrained("lora_model")  # Local saving
# tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model\\tokenizer_config.json',
 'lora_model\\special_tokens_map.json',
 'lora_model\\tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, temperature=0)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
The Eiffel Tower is a famous tall tower in Paris.<|end_of_text|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [25]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if True: model.save_pretrained_merged("model_final_16bit", tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("emaalberti/model_final_16bit", tokenizer, save_method = "merged_16bit", token=os.getenv("HF_TOKEN"))

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [29]:
# Salvataggio locale GGUF
# model.save_pretrained_gguf("model_gguf", tokenizer, quantization_method="q4_k_m")

# Upload GGUF su HuggingFace (opzionale)
# model.push_to_hub_gguf("emaalberti/model_gguf", tokenizer, quantization_method="q4_k_m", token=os.getenv("HF_TOKEN"))

In [1]:
# Save to 8bit Q8_0

# model.save_pretrained("lora_model")
# tokenizer.save_pretrained("lora_model")

# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
# model.push_to_hub("emaalberti/lora_model", token = os.getenv("HF_TOKEN"))

# model.save_pretrained_merged("model", tokenizer, save_method= "merged_16bit")
# model.push_to_hub_merged("emaalberti/lora_model", tokenizer, save_method="merged_16bit", token = os.getenv("HF_TOKEN"))

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
